# Pymkm demo: CO2 hydrogenation to MeOH over Pd-In2O3 catalyst

## MKM class import, object instance and attributes overview

In [2]:
from pymkm.mkm import MicrokineticModel

ModuleNotFoundError: No module named 'pymkm'

In [ ]:
model = MKM('Pd-In2O3', 'rm.mkm', 'g.mkm', t_ref=553)

# Model exploration
To investigate the characteristics of the system under study, several attributes can be easily inspected to check general information like number of elementary reactions, energetics, reaction network, etc.

In [ ]:
print(model)

In [ ]:
model.dh_barrier

In [ ]:
model.df_gibbs

In [ ]:
model.df_system

In [ ]:
model.stoich_numbers

## Thermodynamic consistency analysis

A often overlooked aspect in microkinetic models is its thermodynamic consistency: although we are dealing with catalysis, a kinetic phenomenon, it is strictly bound to thermodynamic constraints. Currently, pymkm allows a quantitative estimation of thermodynamic consistency of microkinetic models: this is done comparing directly the equilibrium constants of the global reaction we are studying with the global constants built from the elementary reactions proposed in the reaction network. This can be easily done using the MKM.thermodynamic_consistency() method, that needs as input only the temperature at which we want to perform the analysis.

In [ ]:
model.thermodynamic_consistency(553)

It is easy to observe from the generated report that the model is not thermodynamically consistent, both at enthalpic and entropic level, as the equilibrium constants of the global reactions calculated with DFT are order of magnitude different from the values extracted from the thermochemistry database. However, considering the MeOH reaction, observing the equilibrium constant one could think the gap is not so huge: this is due to the fact that enthalpy and entropy constants compensate each pther in the opposite direction, "canceling" the discrepancy between database and DFT-derived equilibrium constants.

## Step 2: microkinetic model single run

In [ ]:
model.grl

In [ ]:
exp2 = model.kinetic_run(573, 5e6, [0.8,0.2,0,0,0])

In [ ]:
exp = model.kinetic_run(573, 5e6, [0.8,0.2,0,0,0])

In [ ]:
exp.keys()

In [ ]:
exp["S_MeOH"]

In [ ]:
exp["ddt"]

In [ ]:
exp['r_RWGS']

## Step 3: Apparent activation energy

In [ ]:
model.reactor_model

In [ ]:
Eapp = model.apparent_activation_energy([523,573,20], 5e6, [0.8,0.2,0,0,0], 'MeOH')

In [ ]:
print(Eapp)

## Step 4: Apparent reaction orders

In [ ]:
import numpy as np
composition_matrix = np.array([[0.78,0.22,0,0,0],[0.8,0.2,0,0,0],[0.82,0.18,0,0,0]])

In [ ]:
napp_H2 = model.apparent_reaction_order(573, 5e6, composition_matrix, 'CO2', 'MeOH')

In [ ]:
napp_H2

## Descriptor search: Reversibility analysis

In [ ]:
rev = model.reversibility(573, 5e6, [0.8,0.2,0,0,0])

In [ ]:
rev

## Rate-determining step search: Degree of rate and selectivity control

In [ ]:
output = model.drc_full(573, 5e6, [0.8,0.2,0,0,0], "MeOH")

In [ ]:
output[0]

# Dynamic CSTR

If someone is interested in extracting also values related to conversion and yield, the differential model is not suitable for the purpose. In order to obtain this information, a macroscopic reactor model (i.e., able to provide information about the conversion) is needed. MKM provides the dynamic CSTR model as macroscopic model. In order to use it, the reactor switch must be turned on and information about the reactor set-up must be defined.

In [ ]:
model.set_reactor('dynamic')
model.set_CSTR_params(radius=0.0022,
                      length=0.0022*10,
                      Q=0.66E-6,
                      S_BET=1.74E5,
                      m_cat=1.0E-4)

In [ ]:
x = model.kinetic_run(553, 5e6, [0.8,0.2,0,0,0])

In [ ]:
x.keys()

In [ ]:
x['MASI']

In [ ]:
x['r_RWGS']

In [ ]:
x['r_MeOH']

In [ ]:
x['conversion']

In [ ]:
x['y_out']